## Data Ingestion

In [1]:
import pandas as pd

# column_names = ['id', 'text', 'label']
constraint_train = pd.read_csv("/kaggle/input/dataset-4/constraint_train.csv")
constraint_test = pd.read_csv("/kaggle/input/dataset-4/constraint_test.csv")
constraint_dev = pd.read_csv("/kaggle/input/dataset-4/constraint_dev.csv")

In [2]:
constraint_train.shape, constraint_test.shape, constraint_dev.shape

((4494, 4), (963, 4), (963, 4))

## Data Preparation

In [3]:
LABELS = constraint_train['label'].unique().tolist()
train_label = constraint_train['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/1124859992.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = constraint_train['label'].replace(LABELS, [0, 1]).tolist()


In [4]:
train_data = constraint_train['tweet'].tolist()

In [5]:
dev_data = constraint_dev['tweet'].tolist()
dev_label = constraint_dev['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/2472027419.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = constraint_dev['label'].replace(LABELS, [0, 1]).tolist()


In [6]:
test_data = constraint_test['tweet'].tolist()
test_label = constraint_test['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/1887347934.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = constraint_test['label'].replace(LABELS, [0, 1]).tolist()


In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [8]:
from transformers import BartTokenizer, BartModel, AdamW

max_length = 128

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [10]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [11]:
num_classes = 2

In [12]:
class BARTClassifier(nn.Module):
    def __init__(self, bart_model, num_classes, dropout=0.1):
        super(BARTClassifier, self).__init__()
        self.bart = bart_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bart(input_ids=input_ids, attention_mask=attention_mask)
        
        last_hidden_state = outputs.last_hidden_state  
        
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_hidden_states = torch.sum(last_hidden_state * attention_mask_expanded, 1)
        sum_attention_mask = torch.clamp(attention_mask_expanded.sum(1), min=1e-9)
        pooled_output = sum_hidden_states / sum_attention_mask
        
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [13]:
bart_model = BartModel.from_pretrained('facebook/bart-base')

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [14]:
model = BARTClassifier(bart_model, num_classes)

In [15]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BARTClassifier(
  (bart): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Layer

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    if num_classes == 2:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)
        }
    else:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='weighted'),
            'recall': recall_score(y_true, y_pred, average='weighted'),
            'f1': f1_score(y_true, y_pred, average='weighted')
        }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [17]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [18]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [19]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 141/141 [01:04<00:00,  2.18it/s]



Epoch 1/5
Training Metrics:
Loss: 0.2215
Accuracy: 0.9072
Precision: 0.9072
Recall: 0.8993
F1: 0.9032
Roc_auc: 0.9710

Validation Metrics:
Accuracy: 0.9491
Precision: 0.9974
Recall: 0.8902
F1: 0.9407
Roc_auc: 0.9928


Epoch 2/5: 100%|██████████| 141/141 [01:03<00:00,  2.23it/s]



Epoch 2/5
Training Metrics:
Loss: 0.0856
Accuracy: 0.9693
Precision: 0.9690
Recall: 0.9672
F1: 0.9681
Roc_auc: 0.9952

Validation Metrics:
Accuracy: 0.9761
Precision: 0.9748
Recall: 0.9725
F1: 0.9737
Roc_auc: 0.9963


Epoch 3/5: 100%|██████████| 141/141 [01:03<00:00,  2.23it/s]



Epoch 3/5
Training Metrics:
Loss: 0.0432
Accuracy: 0.9849
Precision: 0.9843
Recall: 0.9843
F1: 0.9843
Roc_auc: 0.9987

Validation Metrics:
Accuracy: 0.9740
Precision: 0.9976
Recall: 0.9451
F1: 0.9706
Roc_auc: 0.9971


Epoch 4/5: 100%|██████████| 141/141 [01:03<00:00,  2.23it/s]



Epoch 4/5
Training Metrics:
Loss: 0.0317
Accuracy: 0.9891
Precision: 0.9912
Recall: 0.9861
F1: 0.9886
Roc_auc: 0.9993

Validation Metrics:
Accuracy: 0.9792
Precision: 0.9860
Recall: 0.9680
F1: 0.9769
Roc_auc: 0.9954


Epoch 5/5: 100%|██████████| 141/141 [01:03<00:00,  2.23it/s]



Epoch 5/5
Training Metrics:
Loss: 0.0307
Accuracy: 0.9904
Precision: 0.9898
Recall: 0.9903
F1: 0.9901
Roc_auc: 0.9992

Validation Metrics:
Accuracy: 0.9740
Precision: 0.9976
Recall: 0.9451
F1: 0.9706
Roc_auc: 0.9965


In [21]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,0.221546,0.907210,0.907226,0.899261,0.903226,0.970952
1,0.085643,0.969292,0.968981,0.967190,0.968085,0.995164
2,0.043168,0.984869,0.984288,0.984288,0.984288,0.998746
3,0.031702,0.989097,0.991175,0.986137,0.988650,0.999312
4,0.030682,0.990432,0.989838,0.990296,0.990067,0.999227


In [22]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.949117,0.997436,0.890160,0.940750,0.992839
1,0.976116,0.974771,0.972540,0.973654,0.996306
2,0.974039,0.997585,0.945080,0.970623,0.997050
3,0.979232,0.986014,0.967963,0.976905,0.995436
4,0.974039,0.997585,0.945080,0.970623,0.996467


In [23]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.9709241952232607,
 'precision': 0.9931350114416476,
 'recall': 0.9455337690631809,
 'f1': 0.96875,
 'roc_auc': 0.9987896393125152}